In [1]:
from functions import *

# 1.Common words

## Firstly, we look for the most pronounced words by each character.

We begin with the two first films:

In [2]:
data_char = pd.read_csv('../Data/temporary_data/Characters_preprocessed.csv', sep=r'\;', encoding='unicode_escape', engine='python')
data_char.Character = data_char.Character.apply(lambda t : string.capwords(t))

In [3]:
data_word = pd.read_csv('../Data/temporary_data/Words_by_character_modified.csv', sep=r'\;', encoding='unicode_escape', engine='python')
data_word = data_word.groupby(['Character']).sum()
# We have to gather again lines because when we had split the lines where more than one character was present, some of them were in 2 lines or more.
data_word = data_word.rename_axis('Character').reset_index()

In [4]:
data_word['Most_common'] = data_word.apply(common_words, axis=1)
data_word.Character = data_word.Character.apply(lambda t : string.capwords(t))

In [5]:
character_all = data_char.Character.values
common_all = []
for char in character_all :
    t  = list(data_word[data_word.Character == char].Most_common.values)
    common_all.append([item for sublist in t for item in sublist])
data_common_word = pd.DataFrame(list(zip(character_all, common_all)), columns = ['Character', 'Most_common'])

Then for the third:

In [6]:
data_char_3 = pd.read_csv('../Data/temporary_data/New_characters.csv', sep=r'\;', encoding='unicode_escape', engine='python')
data_char_3.Character = data_char_3.Character.apply(lambda t : string.capwords(t))

In [7]:
data_word_3 = pd.read_csv('../Data/temporary_data/Words_by_character_3_modified.csv', sep=r'\;', encoding='unicode_escape', engine='python')

In [8]:
data_word_3['Most_common'] = data_word_3.apply(common_words, axis=1)
data_word_3.Character = data_word_3.Character.apply(lambda t : string.capwords(t))

In [9]:
character_3 = data_char_3.Character.values
common_3 = []
for char in character_3 :
    t  = list(data_word_3[data_word_3.Character == char].Most_common.values)
    common_3.append([item for sublist in t for item in sublist])
data_common_word_3 = pd.DataFrame(list(zip(character_3, common_3)), columns = ['Character', 'Most_common'])

## Secondly, we set the most pronounced words from the two first films and the third, and match them to the characters.

### HP1 and HP2

In [10]:
df1 = pd.read_csv('../Data/temporary_data/HP1_preprocessed.csv', sep=';', encoding='unicode_escape')
df2 = pd.read_csv('../Data/temporary_data/HP2_preprocessed.csv', sep=';', encoding='unicode_escape')

In [11]:
df1['Sentence_cleared'] = df1['Sentence_cleared'].apply(lambda t: ast.literal_eval(t))
df2['Sentence_cleared'] = df2['Sentence_cleared'].apply(lambda t: ast.literal_eval(t))

text_1 = df1['Sentence_cleared'].sum()
text_2 = df2['Sentence_cleared'].sum()

We chose to consider the 100 first ones (why? is there another amount to consider ?).  
Notice that there are 9154 overall words and 2224 unique ones.

In [12]:
most_common = Counter((text_1+text_2)).most_common(100)
word_common = [most_common[i][0] for i in range(len(most_common))]

In [13]:
film_common = []
for i in range(len(data_common_word)):
    list_tuple = data_common_word.Most_common[i]
    test = []
    for j in range(len(list_tuple)):
        tuple_word = list_tuple[j]
        word = tuple_word[0]
        if word in word_common:
            test.append(tuple_word)
    film_common.append(test)
data_common_word['In film_common'] = film_common

### HP3

In [14]:
df3 = pd.read_csv('../Data/temporary_data/HP3_preprocessed.csv', sep=';', encoding='unicode_escape')

In [15]:
df3['Sentence_cleared'] = df3['Sentence_cleared'].apply(lambda t: ast.literal_eval(t))

text_3 = df3['Sentence_cleared'].sum()

4287 total words and 1363 unique ones.

In [16]:
most_common_3 = Counter(text_3).most_common(100)
word_common_3 = [most_common_3[i][0] for i in range(len(most_common_3))]

In [17]:
film_common_3 = []
for i in range(len(data_common_word_3)):
    list_tuple_3 = data_common_word_3.Most_common[i]
    test_3 = []
    for j in range(len(list_tuple_3)):
        tuple_word_3 = list_tuple_3[j]
        word_3 = tuple_word_3[0]
        if word_3 in word_common_3:
            test_3.append(tuple_word_3)
    film_common_3.append(test_3)
data_common_word_3['In film_common'] = film_common_3

## Thirdly we build an indicator matrix of the words we want to keep in the model.

Which words do we keep? We select the words related to Harry Potter's world, removing characters' names.

In [18]:
chosen_words = ['professor', 'sir', 'hogwarts', 'slytherin', 'school', 'year', 'gryffindor', 'kill', 'chamber', 'wizard']

### HP1 and HP2

In [19]:
df_film_common = pd.DataFrame(0, columns=chosen_words, index=data_common_word.Character).rename_axis('Character').reset_index()

In [20]:
for i in range(len(df_film_common)):
    list_tuple = data_common_word['In film_common'][i]
    test = []
    for j in range(len(list_tuple)):
        tuple_word = list_tuple[j]
        word = tuple_word[0]
        occurrence = tuple_word[1]
        test.append(tuple_word)
        if word in chosen_words:
            df_film_common[f'{word}'][i] = occurrence

<ipython-input-20-c9b5c6660659>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_film_common[f'{word}'][i] = occurrence


### HP3

In [21]:
df_film_common_3 = pd.DataFrame(0, columns=chosen_words, index=data_common_word_3.Character).rename_axis('Character').reset_index()

In [22]:
for i in range(len(df_film_common_3)):
    list_tuple_3 = data_common_word_3['In film_common'][i]
    test_3 = []
    for j in range(len(list_tuple_3)):
        tuple_word_3 = list_tuple_3[j]
        word_3 = tuple_word_3[0]
        occurrence_3 = tuple_word_3[1]
        test_3.append(tuple_word_3)
        if word_3 in chosen_words:
            df_film_common_3[f'{word_3}'][i] = occurrence_3

<ipython-input-22-58c3e9f5a10f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_film_common_3[f'{word_3}'][i] = occurrence_3


# 2.Gender

## HP1 and HP2

In [23]:
df_gender = data_char[['Character', 'Gender']].copy()

for i in range(len(df_gender)):
    if df_gender.Gender[i] == 'Male':
        df_gender.Gender[i] = 0
    else:
        df_gender.Gender[i] = 1

## HP3

In [24]:
df_gender_3 = data_char_3[['Character', 'Gender']].copy()

for i in range(len(df_gender_3)):
    if df_gender_3.Gender[i] == 'Male':
        df_gender_3.Gender[i] = 0
    else:
        df_gender_3.Gender[i] = 1

# 3.Word count

## HP1 and HP2

In [25]:
df1['Sentence_tokenized'] = df1['Sentence_tokenized'].apply(lambda t: ast.literal_eval(t))
df2['Sentence_tokenized'] = df2['Sentence_tokenized'].apply(lambda t: ast.literal_eval(t))

In [26]:
df1['Word_count'] = df1.apply(word_count, axis=1)
df2['Word_count'] = df2.apply(word_count, axis=1)

In [27]:
num_words_1 = df1.groupby('Character').Word_count.sum()  # summing all words pronounced by each character
num_words_1 = pd.DataFrame(num_words_1).rename_axis('Character').reset_index()

num_words_2 = df2.groupby('Character').Word_count.sum()  # summing all words pronounced by each character
num_words_2 = pd.DataFrame(num_words_2).rename_axis('Character').reset_index()

In [28]:
df_temp = pd.DataFrame(0, columns=['Word count 1', 'Word count 2'], index=data_char.Character).rename_axis('Character').reset_index()

In [29]:
for i in range(len(num_words_1['Character'])):
    for j in range(len(df_temp['Character'])):
        if num_words_1['Character'][i] in df_temp['Character'][j]:
            df_temp['Word count 1'][j] = num_words_1['Word_count'][i]

for i in range(len(num_words_2['Character'])):
    for j in range(len(df_temp['Character'])):
        if num_words_2['Character'][i] in df_temp['Character'][j]:
            df_temp['Word count 2'][j] = num_words_2['Word_count'][i]

<ipython-input-29-b77052397bfe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Word count 1'][j] = num_words_1['Word_count'][i]
<ipython-input-29-b77052397bfe>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Word count 2'][j] = num_words_2['Word_count'][i]


In [30]:
df_temp['Word count'] = df_temp[['Word count 1', 'Word count 2']].sum(axis=1)

total_word = num_words_1['Word_count'].sum() + num_words_2['Word_count'].sum()
df_temp['Word freq(%)'] = df_temp['Word count'].apply(lambda t : (t/total_word)*100)

df_count = df_temp[['Character', 'Word freq(%)']]

## HP3

In [31]:
df3['Sentence_tokenized'] = df3['Sentence_tokenized'].apply(lambda t: ast.literal_eval(t))
df3['Word_count'] = df3.apply(word_count, axis=1)

In [32]:
num_words_3 = df3.groupby('Character').Word_count.sum()  # summing all words pronounced by each character
num_words_3 = pd.DataFrame(num_words_3).rename_axis('Character').reset_index()

In [33]:
df_temp_3 = pd.DataFrame(0, columns=['Word count'], index=data_char_3.Character).rename_axis('Character').reset_index()

In [34]:
for i in range(len(num_words_3['Character'])):
    for j in range(len(df_temp_3['Character'])):
        if num_words_3['Character'][i] in df_temp_3['Character'][j]:
            df_temp_3['Word count'][j] = num_words_3['Word_count'][i]

<ipython-input-34-3245183be488>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_3['Word count'][j] = num_words_3['Word_count'][i]


In [35]:
total_word_3 = num_words_3['Word_count'].sum()
df_temp_3['Word freq(%)'] = df_temp_3['Word count'].apply(lambda t : (t/total_word_3)*100)

df_count_3 = df_temp_3[['Character', 'Word freq(%)']]

# 4.Blood status

## HP1 and HP2

In [36]:
blood_status_col = pd.get_dummies(data_char['Blood status'])
blood_status_mat = pd.concat([data_char['Character'], blood_status_col], axis=1)

In [37]:
blood_status_mat.columns

Index(['Character', 'Half-blood', 'Muggle', 'Muggle-born',
       'Muggle-born or Half-blood', 'Part-Goblin', 'Part-Human or Half-giant',
       'Pure-blood', 'Pure-blood or Half-blood', 'Quarter-Veela', 'Squib',
       'Unknown'],
      dtype='object')

## HP3

In [61]:
blood_status_col_3 = blood_status_mat.columns
blood_status_mat_3 = pd.DataFrame(0, index=np.arange(len(data_char_3['Character'])), columns=blood_status_col_3)
blood_status_mat_3.Character = data_char_3['Character']

In [62]:
for i in range(len(data_char_3)):
    blood = data_char_3['Blood status'][i]
    for j in blood_status_mat_3.columns:
        if j == blood: blood_status_mat_3[j][i] = 1

<ipython-input-62-a37d86c2d28a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if j == blood: blood_status_mat_3[j][i] = 1


# Matrices setup

## HP1 et HP2

In [67]:
my_matrix = data_char[['Character', 'House']]

In [68]:
my_matrix = my_matrix.merge(df_gender, on='Character')
my_matrix = my_matrix.merge(blood_status_mat, on='Character')
my_matrix = my_matrix.merge(df_film_common, on='Character')
my_matrix = my_matrix.merge(df_count, on='Character')

In [69]:
my_matrix.head()

,Character,House,Gender,Half-blood,Muggle,Muggle-born,Muggle-born or Half-blood,Part-Goblin,Part-Human or Half-giant,Pure-blood,Pure-blood or Half-blood,Quarter-Veela,Squib,Unknown,professor,sir,hogwarts,slytherin,school,year,gryffindor,kill,chamber,wizard,Word freq(%)
0,Harry,Gryffindor,0,1,0,0,0,0,0,0,0,0,0,0,10,14,6,7,4,6,0,6,11,4,18.689284
1,Ron,Gryffindor,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,6,2,5,9.352192
2,Hermione,Gryffindor,1,0,0,1,0,0,0,0,0,0,0,0,5,0,4,6,0,3,2,0,4,3,8.229728
3,Dumbledore,Gryffindor,0,1,0,0,0,0,0,0,0,0,0,0,7,0,3,3,4,3,7,0,3,0,7.384104
4,Hagrid,Gryffindor,0,0,0,0,0,0,1,0,0,0,0,0,13,5,9,0,3,4,0,0,0,7,9.910908


In [70]:
# my_matrix.to_csv('../Data/Script_matrix.csv', sep=';', index=False)

## HP3

In [71]:
my_matrix_3 = data_char_3[['Character', 'House']]

In [72]:
my_matrix_3 = my_matrix_3.merge(df_gender_3, on='Character')
my_matrix_3 = my_matrix_3.merge(blood_status_mat_3, on='Character')
my_matrix_3 = my_matrix_3.merge(df_film_common_3, on='Character')
my_matrix_3 = my_matrix_3.merge(df_count_3, on='Character')

In [73]:
my_matrix_3.head()

,Character,House,Gender,Half-blood,Muggle,Muggle-born,Muggle-born or Half-blood,Part-Goblin,Part-Human or Half-giant,Pure-blood,Pure-blood or Half-blood,Quarter-Veela,Squib,Unknown,professor,sir,hogwarts,slytherin,school,year,gryffindor,kill,chamber,wizard,Word freq(%)
0,Sirius,Gryffindor,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,4,0,0,5.262591
1,Lupin,Gryffindor,0,1,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,15.400344
2,Pettigrew,Gryffindor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.194576
3,Parvati Patil,Gryffindor,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.010762
4,Trelawney,Ravenclaw,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,3.153250


In [74]:
# my_matrix_3.to_csv('../Data/Script_matrix_3.csv', sep=';', index=False)